# Semantic Music Dataset Maker (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2024

***

# (SETUP ENVIRONMENT)

In [ ]:
# @title Install dependecies
!git clone --depth 1 https://github.com/asigalov61/tegridy-tools
!pip install datasets -U

In [ ]:
#@title Import all needed modules

print('=' * 70)
print('Loading needed modules. Please wait...')

import os
import copy
import math
import statistics
import random
import pickle
import shutil
import re

from collections import Counter

from joblib import Parallel, delayed, parallel_config

#from tqdm import tqdm
import tqdm

import matplotlib.pyplot as plt

print('=' * 70)
print('Creating I/O dirs...')

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

print('=' * 70)
print('Loading TMIDIX module...')

%cd /content/tegridy-tools/tegridy-tools

import TMIDIX

%cd /content/

from datasets import load_dataset

print('=' * 70)

# (DOWNLOAD SOURCE LYRICS DATASET)

In [ ]:
# @title Small lyrics dataset (recommended)

#======================================================================
# Helper function
#======================================================================

def clean_string(input_string):
  input_string = input_string.replace('\'', '').replace('-', ' ')
  str1 = re.sub(r'[^a-z ]', '', input_string.lower().strip())
  str2 = re.sub(' +', ' ', str1)
  return str2.strip()

#======================================================================

print('=' * 70)
print('Downloading small lyrics dataset...')
print('=' * 70)
lyrics_small = load_dataset("tsterbak/lyrics-dataset")

print('Done!')
print('=' * 70)

print('Prepping the dataset...')
print('=' * 70)

lyrics_small_clean = []

for l in tqdm.tqdm(lyrics_small['train']):
  lyrics_small_clean.append(l['seq'])

random.shuffle(lyrics_small_clean)

print('=' * 70)

lyrics_sentences_all = set()

str_counter = 0

for l in tqdm.tqdm(lyrics_small_clean):
  ls = l.split('\n')
  str_counter += len(ls)
  for s in ls:
    lyrics_sentences_all.add(clean_string(s))

print('=' * 70)

lyrics_words_sets = [set() for _ in range(256)]
lyrics_sets = [set() for _ in range(256)]

for l in tqdm.tqdm(lyrics_sentences_all):
  ls = l.strip().split(' ')
  if len(ls) < 257:
    if l != '':
      lyrics_sets[len(ls)-1].add(l.strip())
    for i, s in enumerate(ls):
      if s != '':
        lyrics_words_sets[i].add(s.strip())

print('=' * 70)

lyrics_words_lists = []

for l in tqdm.tqdm(lyrics_words_sets):
  lyrics_words_lists.append(sorted(l))

print('=' * 70)

lyrics_sets_lists = []

for l in tqdm.tqdm(lyrics_sets):
  lst = sorted(l)
  lyrics_sets_lists.append(lst)

print('=' * 70)

lyrics_lists = []

for l in tqdm.tqdm(lyrics_sets):
  lyrics_lists.append(sorted(l))

print('Done!')
print('=' * 70)
print('Dataset is ready!')
print('=' * 70)

# (DOWNLOAD SOURCE MIDI DATASET)

In [ ]:
# @title POP909 MIDI Dataset (recommended)
!git clone --depth 1 https://github.com/music-x-lab/POP909-Dataset

In [ ]:
# @title ASAP MIDI Dataset
!git clone --depth 1 https://github.com/fosfrancesco/asap-dataset

# (PROCESS MIDIs)

In [ ]:
#@title Prep MIDI file list
source_MIDI_dataset_path = "/content/POP909-Dataset" # @param {type:"string"}

print('=' * 70)
print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

filez = list()
for (dirpath, dirnames, filenames) in os.walk(source_MIDI_dataset_path):
    filez += [os.path.join(dirpath, file) for file in filenames if file.endswith('.mid')]
print('=' * 70)

if not filez:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

else:
  print('Randomizing file list...')
  random.shuffle(filez)
  print('Done!')
  print('=' * 70)
  print('Total files:', len(filez))
  print('=' * 70)

In [ ]:
#@title Process MIDIs with TMIDIX MIDI processor

print('=' * 70)
print('TMIDIX MIDI Processor')
print('=' * 70)
print('Starting up...')
print('=' * 70)

all_scores = []

print('Processing MIDI files. Please wait...')
print('=' * 70)

for f in tqdm.tqdm(filez):

    try:

      label = os.path.basename(f).split('.mid')[0]

      raw_score = TMIDIX.midi2single_track_ms_score(open(f, 'rb').read())

      escore = TMIDIX.advanced_score_processor(raw_score, return_enhanced_score_notes=True)[0]

      escore = TMIDIX.augment_enhanced_score_notes(escore, timings_divider=32)

      dscore = TMIDIX.delta_score_notes(escore)

      cscore = TMIDIX.chordify_score([d[1:] for d in dscore])

      src_score = []

      sscore = []

      time = 0
      ptime = 0

      for c in cscore:

        chord = []

        delta_time = max(0, min(127, c[0][0]))

        chord.append(delta_time)

        for e in c:

          dur =  max(1, min(127, e[1]))
          cha = max(0, min(15, e[2]))
          ptc = max(1, min(127, e[3]))
          vel = max(1, min(127, e[4]))
          pat = max(1, min(127, e[5]))

          chord.extend([dur, ptc])

        sscore.append(chord)

        if time >= ptime + 128 and sscore:
          src_score.append(sscore)
          sscore = []
          ptime = time

        time += delta_time

      if src_score:
        all_scores.append([label, src_score])

    except KeyboardInterrupt:
        print('Saving current progress and quitting...')
        break

    except Exception as ex:
        print('WARNING !!!')
        print('=' * 70)
        print(f)
        print('=' * 70)
        print('Error detected:', ex)
        print('=' * 70)
        continue

print('Done!')
print('=' * 70)
print('Enjoy!')
print('=' * 70)

In [ ]:
#@title Save processed MIDIs
TMIDIX.Tegridy_Any_Pickle_File_Writer(all_scores, '/content/Processed_MIDIs')

In [ ]:
# @title Load processed MIDIs
all_scores = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/Processed_MIDIs')

# (PREP PROCESSED MIDIs)

In [ ]:
# @title Prep processed MIDIs

print('=' * 70)
print('Prepping processed MIDIs...')
print('=' * 70)

random.shuffle(all_scores)

all_scores_ext = []

for aa in tqdm.tqdm(all_scores):
  for a in aa[1]:
    all_scores_ext.extend(a)

print('Done!')
print('=' * 70)

print('Creating scores sets...')

all_scores_set = set(tuple(s) for s in all_scores_ext)

all_scores_list = [list(a) for a in all_scores_set]

all_scores_set = sorted(all_scores_set, key=lambda x: (len(x), x))

print('Done!')
print('=' * 70)

selected_lyrics = []

lyrics_lists_idxs = [0] * 256

for s in tqdm.tqdm(all_scores_set):
  sl = len(s)
  lyr_list = lyrics_lists[sl-1]
  lyr = lyr_list[lyrics_lists_idxs[sl-1]]
  lyrics_lists_idxs[sl-1] += 1
  selected_lyrics.append([list(s), lyr, len(lyr.split())])

selected_lyrics = sorted(selected_lyrics, key=lambda x: (x[2], x[1]))

print('=' * 70)

selected_lyrics_ext = ''

for s in tqdm.tqdm(selected_lyrics):
  selected_lyrics_ext += s[1] + ' '

print('=' * 70)

words_dict = list(set(selected_lyrics_ext.split(' ')))

print('Done!')
print('=' * 70)

# (CONVERT)

In [ ]:
# @title Convert processed MIDIs to semantic music scores

print('=' * 70)
print('Converting MIDIs to semantic music scores...')
print('Please wait...')
print('=' * 70)

all_scores_lyrics = []

chords = [tuple(c[0]) for c in selected_lyrics]

for ccc in tqdm.tqdm(all_scores):
  sslyr = []
  fn = ccc[0]
  for cc in ccc[1]:
    slyr = []
    for c in cc:
      lyr = selected_lyrics[chords.index(tuple(c))][1]
      slyr.append(lyr)
    sslyr.append(slyr)

  all_scores_lyrics.append([fn, sslyr])

print('Done!')
print('=' * 70)

In [ ]:
# @title Save sematic music dataset
TMIDIX.Tegridy_Any_Pickle_File_Writer([all_scores, all_scores_lyrics, selected_lyrics, words_dict], '/content/Semantic_Music_Dataset')

In [ ]:
# @title Load semantic music dataset
all_scores, all_scores_lyrics, selected_lyrics, words_dict = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/Semantic_Music_Dataset')

# (TEST SEMANTIC DATASET)

In [ ]:
# @title Test semantic score dataset
print('=' * 70)
print('Loading random score...')

semantic_score = random.choice(all_scores_lyrics)
chords_dict = [s[1] for s in selected_lyrics]

file_name = semantic_score[0]
sem_score = semantic_score[1]

print('Done!')
print('=' * 70)

print('Reconstructing score...')
print('=' * 70)

delta_chords_score = []

for s in tqdm.tqdm(sem_score):
  for ss in s:

    idx = chords_dict.index(ss)
    chord = selected_lyrics[idx][0]

    delta_chords_score.append(chord)

print('Done!')
print('=' * 70)
#===============================================================================

song_f = []

time = 0
dur = 1
vel = 90
pitch = 60
channel = 0

for s in delta_chords_score:

  time += s[0] * 32

  for i in range(1, len(s), 2):

    dur = s[i] * 32
    ptc = s[i+1]

    vel = max(40, ptc)

    song_f.append(['note', time, dur, channel, ptc, vel, 0])

data = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                output_signature = file_name,
                                                output_file_name = '/content/Semantic-Music-Composition',
                                                track_name='Project Los Angeles',
                                                )

print('=' * 70)

# Congrats! You did it! :)